# Introduction to Chains in LangChain

In [3]:
from langchain.llms import OpenAI

llm = OpenAI()

llm.predict("Hi")

",\n\nI'm sorry to hear that you're having trouble getting your computer to start up. This is a common issue and there are a few things that you can try to get your computer back up and running.\n\nFirst, you'll want to check the power cords and make sure they are plugged in correctly. You might also want to try a different power outlet to see if that makes a difference.\n\nIf that doesn't work, you may need to reset your computer. To do this, turn off your computer, unplug the power cord, and remove the battery (if applicable). Then press the power button to drain any remaining power. Wait a few minutes and then plug the power cord back in and turn on the computer.\n\nIf the above steps don't work, then you may need to take your computer to a professional for help. They'll be able to diagnose the issue and get your computer up and running again.\n\nI hope this helps!"

In [10]:
llm.invoke("Hi")

',\n\nI am using a FMCOMMS2/3 board. I want to use the board as a transmitter. I have connected the RF DACs and the splitters and have the board set up with the ADI reference HDL design. I have also designed my own HDL design.\n\nMy question is, how do I configure the FMCOMMS2/3 board to transmit? What are the steps that I need to take in order to get the FMCOMMS2/3 board to transmit?\n\nThanks!'

In [4]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI()

chat_model.predict("Hi")

# or

chat_model.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?')

In [15]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?')

Understanding diff between chatmodel output and llm model output

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

llm = OpenAI()
chat_model =  ChatOpenAI()

In [18]:
from langchain.schema import HumanMessage

text = "What is a good hobby to develop to be more creative?"

messages = [HumanMessage(content=text)]

llm.invoke(text)

chat_model.invoke(messages)

AIMessage(content='There are numerous hobbies that can help develop creativity. Here are a few suggestions:\n\n1. Painting or drawing: Engaging in visual arts can unleash your creativity and allow you to explore different techniques, colors, and styles.\n\n2. Writing or journaling: Expressing your thoughts, feelings, or ideas through writing can enhance your creativity and help you develop a unique voice.\n\n3. Photography: Capturing images and experimenting with composition, lighting, and editing can stimulate your creativity and train your eye for detail.\n\n4. Playing a musical instrument: Learning to play an instrument or composing music can tap into your creative side and provide an outlet for self-expression.\n\n5. Crafting or DIY projects: Engaging in activities like knitting, woodworking, or creating handmade crafts allows you to experiment with various materials and techniques, fostering creativity.\n\n6. Cooking or baking: Exploring new recipes, experimenting with flavors, an

Prompt Template basics

- advatange of abstracting partial variables that go into a prompt string which makes this format a better option with respect to just a regular string.

See: https://python.langchain.com/docs/modules/model_io/prompts

In [20]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good hobby to develop to be more {activity}?")

prompt.format(activity="creative") 

'What is a good hobby to develop to be more creative?'

ChatPromptTemplate

In [25]:
from langchain.prompts import ChatPromptTemplate

template_prompt = "You are a helpful assistant that will translate {input_lang} to {output_lang}."
human_prompt = "{input_text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template_prompt), 
    ("human", human_prompt)
])

chat_prompt.format_messages(input_lang="English", output_lang="Portuguese", input_text="Hi! I am learning the 'langchain' framework")

[SystemMessage(content='You are a helpful assistant that will translate English to Portuguese.'),
 HumanMessage(content="Hi! I am learning the 'langchain' framework")]

# [Output Parser](https://python.langchain.com/docs/get_started/quickstart#:~:text=for%20more%20detail.-,output%20parsers)


3 Types
- LLM text into structured informatiob (e.g JSON)
- ChatMessage into string
- Extra info (e.g. OpenAI function invocation) into string

In [26]:
from langchain.schema import BaseOutputParser
from typing import List


class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""
    def parse(self, text: str) -> List[str]:
        return text.strip().split(",")
    
    
CommaSeparatedListOutputParser().parse("I, love, learning, new, and, cool, frameworks")

['I', ' love', ' learning', ' new', ' and', ' cool', ' frameworks']

# Intro to the LCEL language

Composing with [LCEL](https://python.langchain.com/docs/expression_language)

Now, let's talk chains!

In [28]:
# Old standard

from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({"text": "colors"})
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['red', 'blue', 'green', 'yellow', 'orange']

Multiple Chains

In [1]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

'El país en el que nació la ciudad de Honolulu, Hawái, donde nació Barack Obama, el 44º presidente de los Estados Unidos, es Estados Unidos.'

Runable interface

In [2]:
from langchain.schema.runnable import RunnableMap, RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [3]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of There is no fruit specifically named "Coral." and the flag of Comoros?')])

In [5]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color of coral is a pinkish-orange hue, resembling the color of the marine organism known as coral. \n\nThe flag of Comoros consists of four horizontal stripes. From top to bottom, the stripes are yellow, white, red, and blue. In the upper hoist corner, there is a green isosceles triangle with a white crescent and four white stars inside it.')

In [6]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [7]:
chain.invoke({"input": "scrum"})

'While Scrum has many benefits, it is important to acknowledge the potential downsides and limitations of the framework. The complexity and learning curve associated with Scrum can be challenging, especially for teams new to agile methodologies. It requires a significant commitment of time and effort, as regular meetings and collaboration are integral to the process. Additionally, the flexible and iterative nature of Scrum may make it difficult to predict and plan for project timelines and deliverables, and it may require a skilled Scrum Master to ensure successful implementation. \n\nFurthermore, the emphasis on teamwork and collaboration may present challenges if team members do not work well together or have conflicting interests. The lack of comprehensive documentation and potential for scope creep can also be concerns for certain projects. Additionally, the cultural shift required to implement Scrum and the potential resistance to change from team members accustomed to traditional

# [Deployment with LangServe](https://python.langchain.com/docs/get_started/quickstart#:~:text=LangSmith%20head%20here.-,serving%20with%20langserve)

Deploying with langserve involves:

1. Define the chain 
2. Define a fastapi app
3. Define a route from which to serve the chain `langserve.add_routes`

In [ ]:
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langserve import add_routes

# 1. Chain definition

class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
category_chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()

# 2. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

# 3. Adding chain route
add_routes(
    app,
    category_chain,
    path="/category_chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

[Client](https://python.langchain.com/docs/get_started/quickstart#:~:text=try%20it%20out!-,client)

Now let's set up a client for programmatically interacting with our service. We can easily do this with the langserve.RemoteRunnable. Using this, we can interact with the served chain as if it were running client-side.

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/category_chain/")
remote_chain.invoke({"text": "colors"})
# >> ['red', 'blue', 'green', 'yellow', 'orange']